비밀번호 등 관리하는 법
config.py 생성 후

DB_host = '127.0.0.1'
DB_port = 3307
DB_user = 'root'
DB_pwd = 'test1234'
DB_db = 'test'

:wq


이렇게 만든다음

newsdao에서 import 하도록

import config as cfg


connection_string = "asdaasdasd://{}:{}".format(cfg.DB_user,..)

이런식으로 불러옴


vi .gitignore
config.py 

이렇게 config.py는 서버에 올라가지 않고 로컬에만 존재하도록


# MongoDB

- 스키마 필요없음
- document 자체를 저장할 수 있고, 다른 document를 임베딩한 것도 넣을 수 있다.
- RDB와 달리 join, transaction 등 지원하지 않음. 최근에는 지원하기도

## 학습목표

 1. mongodb 설치
 2. mongodb CRUD
 3. mongoimport / mongoexport 사용
 3. mongodb aggregation 이해 및 숙지
 4. pymongo 모듈을 통한 실습

### mongodb 설치
 * https://docs.mongodb.com/manual/installation/
 * community edition 설치
 
 * terminal에서 아래의 명령어 차례대로 수행
   * sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv 0C49F3730359A14518585931BC711F9BA15703C6
   * lsb_release -a  (ubuntu 버젼 확인)
   * ubuntu 12.04의 경우
     * echo "deb [ arch=amd64 ] http://repo.mongodb.org/apt/ubuntu precise/mongodb-org/3.4 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-3.4.list
   * ubuntu 14.04의 경우
     * echo "deb [ arch=amd64 ] http://repo.mongodb.org/apt/ubuntu trusty/mongodb-org/3.4 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-3.4.list
   * ubuntu 16.04의 경우
     * echo "deb [ arch=amd64,arm64 ] http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.4 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-3.4.list
   * sudo apt-get update
   * sudo apt-get install -y mongodb-org
   
   * sudo vi /etc/mongod.conf
     * bindIp: 127.0.0.1 -> 0.0.0.0으로 변경 (로컬에서 서버로)
    
   * sudo service mongod start or restart
   * sudo service mongod status 로 확인하였을 때 active면 잘된 것.
   * ctrl+c로 나가서 mongo입력하여 mongo에 접속
   

## mongodb 삭제
 * sudo apt-get purge mongodb-org*

* 설치 확인
 - 터미널에서 mongo 입력
 - mongo shell이 뜨는지 확인

* local에서 접속 확인
  * local에서 mongo 설치 (client)
    * Mac OSX 
      - brew install mongo
    * Windows 
      - https://www.mongodb.com/download-center?jmp=nav#community
      - msi installer download
      - path에 설치된 경로 추가 후 
      
  * mongo --host AWS_HOST --port 27017

### Why mongodb?
 * rdb의 경우, 테이블 구조에 프로그램 객체를 맞추는 문제로 인해 개발 생산성이 떨어짐
   - ORM 기술이 있긴 하지만, 개발자 입장에서 여전히 부자연 스러움
 * scheme 미리 정해져 있지 않기 때문에, 개발 생산성이 좋음
 * join, transaction을 지원하지 않기 때문에, 속도가 빠름


### mongodb의 데이터 구성
 - db, collection으로 구성
 - 데이터는 collection의 document 형식(python dictionary)으로 저장 됨
 - collection들의 논리적인 집합이 database
 
### database 선택
 - show dbs - 전체 데이터베이스 열람
 - use [DB 이름] - 데이터베이스 선택
   * e.g) use test - test 데이터베이스 선택
   
 
 - show collections - 선택된 데이터베이스의 콜렉션 열람
 - db.[Collection 이름].함수() 로 해당 컬렉션의 데이터 열람 및 조작
   * db를 고른 이후에는 db라는 이름으로 쓴다.
   * e.g) db.actors.find() - actors 컬렉션에서 검색
          db.actors.insertOne() - actors 컬렉션에서 삽입

RDB는 테이블 구조에 내 데이터를 맞춰야하는데 
몽고는 스키마 없이 그냥 내가 원하는 대로, 넣을 수 있음.

* Mongodb CRUD (deprecated)
   - insert
     * db.person.insert({ "name" : "aaron", "age" : 31, "regdate" : new Date() })
     
   - update
     *  db.person.update({ name : "aaron" }, { $set : { gender : "M" }})
     
   - find / findOne
   
     * db.person.find()
     * db.person.findOne()
     * db.person.find({ name : "aaron" })
     * db.person.find({ name : "aaron", or : [{ age : 30 }, { age : 21 }] })
     * db.person.find({ name : "aaron", nor : [{ age : 20 }, { age : 21 }] })
     * db.person.find({ age : { in : [20, 21] }})
     * db.person.find({ age : { nin : [20, 21] }})
     * db.person.findOne({name : 'Aaron'}, {name : 1, regdate : 1})
     * db.person.find({ name : null })
     * db.person.find().sort({ name : 1 }) # DESC
     * db.person.find().sort({ name : -1 }) # ASC
     * db.person.find().limit(2)
     * db.person.find().limit(2).skip(2) # 2건너 뛰고 2개 가져옴
     * db.person.find({ name : { $exists : false } }) # 특정 필드가 없는 문제 가져옴
   - count
     * db.person.count()
     * db.person.find({ name : "aaron" }).count()
   - remove
     * db.person.remove({ name : "aaron" })
   - drop - collection 삭제
     * db.person.drop()

* mongodb CRUD 
  - insertOne, insertMany
    - insertOne - 한개의 document 생성
    - insertMany - list of document 생성
  - findOne, find
    - findOne - 매칭되는 한개의 document 검색
    - find    - 매칭되는 list of document 검색 
  - updateOne, updateMany
    - updateOne - 매칭되는 한개의 document 업데이트
    - updateMany - 매칭되는 list of document 업데이트
  - removeOne, removeMany
    - removeOne - 매칭되는 한개의 document 삭제
    - removeMany - 매칭되는 list of document 삭제
  

### mongo shell
 - 로컬에서 서버가 돌아갈 경우,
   - mongo
 - 원격 서버에 접속할 경우 
   - mongo --host 'host_address' --port 'port'
   - e.g) mongo --host 1111.111.11.111 --port 27017


### pymongo
 - mongodb python module
 - https://api.mongodb.com/python/current/
 - pip install pymongo

In [1]:
mongo_server = '35.167.74.63' #내 IP
import pymongo

이제 DB에 추가하는 실습을 해보자.
- MySQL은 존재하는 테이블에만 넣을 수 있다면 mongodb는 동적으로 생성하여 넣을 수 있다.

### insertion 예제

* insert one

In [3]:
# insert_one으로 하나씩 삽입
import requests
import re
import datetime
from bs4 import BeautifulSoup
from pymongo import MongoClient


def get_hot_actors():
    cine_url = 'http://www.cine21.com/rank/person/content'
    
    # mongo driver 생성
    mongo = MongoClient(mongo_server, 27017)
    
    # cine21 database -> actor collection 를 지정
    db = mongo.cine22
    actor = db.actor
    #actor = mongo.cine21.actor
    
    data1 = {}
    data1['section'] = 'actor'
    data1['period_start'] = '2016-09'
    data1['gender'] = 'all'
    
    actors = []
    for i in xrange(1, 5):
        data1['page'] = i

        res = requests.post(cine_url, data = data1)
        
        soup = BeautifulSoup(res.text)
        # 이렇게 바로 utf-8으로 설정 할 수 도 있음
        #soup = BeautifulSoup(res.content, from_encoding='utf-8')

        names = soup.find_all('div', attrs = {'class' : 'name'})
        for name in names:
            actor_name = name.get_text()
            actor_name = re.sub('\(.+\)', '', actor_name)
            name = actor_name.encode('utf-8')
            print name,
            actor.insert_one({"actor_name" : name, 
                              'datetime' : datetime.datetime.now()}) # mongodb에서는 dict가 아니라 document라고 부른다.
            
get_hot_actors()

유해진 송강호 이동휘 공유 조정석 한지민 현빈 이병헌 이준 정우성 엄지원 강동원 김주혁 조윤희 신성록 장영남 도경수 임지연 김우빈 김남길 김대명 이해영 조인성 오달수 정진영 엄태구 문정희 김아중


* insert many

In [8]:
# insert_many로 한꺼번에 삽입
import requests
import re
from bs4 import BeautifulSoup
from pymongo import MongoClient


def get_hot_actors():
    cine_url = 'http://www.cine21.com/rank/person/content'
    
    # mongo driver 생성
    mongo = MongoClient(mongo_server, 27017)
    
    # cine21 database -> actor collection
    #actor = mongo.cine21.actor
    db = mongo.cine22
    actor = db.actor2
    
    
    data1 = {}
    data1['section'] = 'actor'
    data1['period_start'] = '2016-09'
    data1['gender'] = 'all'
    
    actors = []
    for i in xrange(1, 5):
        data1['page'] = i

        res = requests.post(cine_url, data = data1)
        
        soup = BeautifulSoup(res.text)
        # 이렇게 바로 utf-8으로 설정 할 수 도 있음
        #soup = BeautifulSoup(res.content, from_encoding='utf-8')

        names = soup.find_all('div', attrs = {'class' : 'name'})
        for name in names:
            actor_name = name.get_text()
            actor_name = re.sub('\(.+\)', '', actor_name)
            name = actor_name.encode('utf-8')
            print name,
            actors.append({'name' : name})
            
    # insert_many를 이용하여 list를 한꺼번에 삽입
    actor.insert_many(actors)
            
get_hot_actors()

 유해진 송강호 이동휘 공유 조정석 한지민 현빈 이병헌 이준 정우성 엄지원 강동원 김주혁 조윤희 신성록 장영남 도경수 임지연 김우빈 김남길 김대명 이해영 조인성 오달수 정진영 엄태구 문정희 김아중


* list, dict insertion
 - list와 dictionary로 삽입 가능
 - 빠른 개발 생산성과 직결됨

In [10]:
mongo = MongoClient(mongo_server, 27017)
movie = mongo.test.movie

# 리스트, 객체 삽입 가능
movie.insert_one({'title' : '암살', 'castings' : ['이정재', '전지현', '하정우']})
movie.insert_one({'title' : '실미도', 'castings' : ['설경구', '안성기'], 
                              'datetime' : {'year' : '2003', 'month' : 3}}) #document embeding을 무한정 할 수 있다.
#shell에서 확인: use movie -> db.movie.find()

In [46]:
data = []
data.append({'name' : 'aaron', 'age' : 20})
data.append({'name' : 'bob', 'age' : 30})
data.append({'name' : 'cathy', 'age' : 25})
data.append({'name' : 'david', 'age' : 27})
data.append({'name' : 'erick', 'age' : 28})
data.append({'name' : 'fox', 'age' : 32})
data.append({'name' : 'hmm'})

tmp = mongo.test.temp
tmp.insert_many(data)

In [11]:
data = []
data.append({'name' : 'aaron', 'age' : 20})
data.append({'name' : 'bob', 'age' : 30})
data.append({'name' : 'cathy', 'age' : 25})
data.append({'name' : 'david', 'age' : 27})
data.append({'name' : 'erick', 'age' : 28})
data.append({'name' : 'fox', 'age' : 32})
data.append({'name' : 'hmm'})

tmp = mongo.test.temp2
tmp.insert_many(data)

### Find 예제

* find_one, find

In [24]:
# findOne으로 찾기
mongo = MongoClient(mongo_server, 27017)
actor = mongo.cine21.actor

result = actor.find_one({'actor_name' : '변영효'})
print result

result = actor.find_one({'actor_name' : '김원'})
print result

result = actor.find_one({'actor_name' : '송강호'})
print result # 있는 경우 document를 반환함

# find 의 경우 cursor 반환
result = actor.find({'actor_name' : '송강호'})
for record in result:
    print record
    print record['actor_name']
    
print
result = actor.find()
for record in result:
    print record['actor_name']

None
None
{u'actor_name': u'\uc1a1\uac15\ud638', u'_id': ObjectId('58c203d66c42db2b20dc9985'), u'datetime': datetime.datetime(2017, 3, 10, 10, 39, 34, 411000)}
{u'actor_name': u'\uc1a1\uac15\ud638', u'_id': ObjectId('58c203d66c42db2b20dc9985'), u'datetime': datetime.datetime(2017, 3, 10, 10, 39, 34, 411000)}
송강호

유해진
송강호
이동휘
공유
조정석
한지민
현빈
이병헌
이준
정우성
엄지원
강동원
김주혁
조윤희
신성록
장영남
도경수
임지연
김우빈
김남길
김대명
이해영
조인성
오달수
정진영
엄태구
문정희
김아중


* sort
 - mysql의 order by에 해당
 - find로 데이터 열람 시, 정렬 조건 명시 가능

In [25]:
import pymongo

# 기본적으로 오름차순
result = actor.find().sort('actor_name')
for record in result:
    print record
    
result = actor.find().sort('actor_name', pymongo.ASCENDING) # pymongo에서 오름차순, 내림차순 할때
for record in result:
    print record

# 내림차순 명시
result = actor.find().sort('actor_name', pymongo.DESCENDING)
for record in result:
    print record
    

# sort by multiple fields
result = actor.find().sort([('actor_name', pymongo.ASCENDING),
                            ('datetime', pymongo.DESCENDING)]) #기준이 여러가지인 경우 tuple list로 주면 됨. 먼저 써준 기준이 먼저 적용됨

# 한글이름이면 ㄱ,ㄴ,ㄷ 순

for record in result:
    print record

{u'actor_name': u'\uac15\ub3d9\uc6d0', u'_id': ObjectId('58c203d76c42db2b20dc998f'), u'datetime': datetime.datetime(2017, 3, 10, 10, 39, 35, 908000)}
{u'actor_name': u'\uacf5\uc720', u'_id': ObjectId('58c203d66c42db2b20dc9987'), u'datetime': datetime.datetime(2017, 3, 10, 10, 39, 34, 665000)}
{u'actor_name': u'\uae40\ub0a8\uae38', u'_id': ObjectId('58c203d96c42db2b20dc9997'), u'datetime': datetime.datetime(2017, 3, 10, 10, 39, 37, 130000)}
{u'actor_name': u'\uae40\ub300\uba85', u'_id': ObjectId('58c203d96c42db2b20dc9998'), u'datetime': datetime.datetime(2017, 3, 10, 10, 39, 37, 257000)}
{u'actor_name': u'\uae40\uc544\uc911', u'_id': ObjectId('58c203da6c42db2b20dc999f'), u'datetime': datetime.datetime(2017, 3, 10, 10, 39, 38, 377000)}
{u'actor_name': u'\uae40\uc6b0\ube48', u'_id': ObjectId('58c203d96c42db2b20dc9996'), u'datetime': datetime.datetime(2017, 3, 10, 10, 39, 37, 1000)}
{u'actor_name': u'\uae40\uc8fc\ud601', u'_id': ObjectId('58c203d86c42db2b20dc9990'), u'datetime': datetime.d

* 필드값이 존재하는 경우 검색

In [26]:
for doc in tmp.find({'age' : {'$exists' : False}}): # age가 존재하지 않는 것만 가져와라
    print doc

print 
for doc in tmp.find({'age' : {'$exists' : True}}):
    print doc

{u'_id': ObjectId('58c208bd6c42db2b20dc99ea'), u'name': u'hmm'}

{u'age': 20, u'_id': ObjectId('58c208bd6c42db2b20dc99e4'), u'name': u'aaron'}
{u'age': 30, u'_id': ObjectId('58c208bd6c42db2b20dc99e5'), u'name': u'bob'}
{u'age': 25, u'_id': ObjectId('58c208bd6c42db2b20dc99e6'), u'name': u'cathy'}
{u'age': 27, u'_id': ObjectId('58c208bd6c42db2b20dc99e7'), u'name': u'david'}
{u'age': 28, u'_id': ObjectId('58c208bd6c42db2b20dc99e8'), u'name': u'erick'}
{u'age': 32, u'_id': ObjectId('58c208bd6c42db2b20dc99e9'), u'name': u'fox'}


* 필드의 범위로 검색

* gt, gte
 - gt : 크다, gte : 크거나 같다


* lt, lte
 - lt : 작다, lte : 작거나 같다

In [27]:
for doc in tmp.find({'age' : {'$gte' : 27}}): # 27살 이상
    print doc

{u'age': 30, u'_id': ObjectId('58c208bd6c42db2b20dc99e5'), u'name': u'bob'}
{u'age': 27, u'_id': ObjectId('58c208bd6c42db2b20dc99e7'), u'name': u'david'}
{u'age': 28, u'_id': ObjectId('58c208bd6c42db2b20dc99e8'), u'name': u'erick'}
{u'age': 32, u'_id': ObjectId('58c208bd6c42db2b20dc99e9'), u'name': u'fox'}


In [28]:
for doc in tmp.find({'age' : {'$gte' : 27}}):
    print doc

{u'age': 30, u'_id': ObjectId('58c208bd6c42db2b20dc99e5'), u'name': u'bob'}
{u'age': 27, u'_id': ObjectId('58c208bd6c42db2b20dc99e7'), u'name': u'david'}
{u'age': 28, u'_id': ObjectId('58c208bd6c42db2b20dc99e8'), u'name': u'erick'}
{u'age': 32, u'_id': ObjectId('58c208bd6c42db2b20dc99e9'), u'name': u'fox'}


* or, nor

In [29]:
for doc in tmp.find({'$or' : [{'age' : 27}, {'age' : 30}]}):
    print doc

{u'age': 30, u'_id': ObjectId('58c208bd6c42db2b20dc99e5'), u'name': u'bob'}
{u'age': 27, u'_id': ObjectId('58c208bd6c42db2b20dc99e7'), u'name': u'david'}


In [30]:
# name이 aaron이고, age가 20이거나 30인 doc
for doc in tmp.find({'name' : 'aaron', '$or' : [{'age' : 20}, {'age' : 30}]}):
    print doc

{u'age': 20, u'_id': ObjectId('58c208bd6c42db2b20dc99e4'), u'name': u'aaron'}


In [31]:
# name이 aaron이고, age가 20이하 이거나 30인 doc
for doc in tmp.find({'name' : 'aaron', '$or' : [{'age' : {'$lte':20}}, {'age' : 30}]}):
    print doc

{u'age': 20, u'_id': ObjectId('58c208bd6c42db2b20dc99e4'), u'name': u'aaron'}


In [32]:
for doc in tmp.find({'$or' : [{'age' : {'$gte' : 29}}, {'age' : {'$lte' : 25}}]}):
    print doc

{u'age': 20, u'_id': ObjectId('58c208bd6c42db2b20dc99e4'), u'name': u'aaron'}
{u'age': 30, u'_id': ObjectId('58c208bd6c42db2b20dc99e5'), u'name': u'bob'}
{u'age': 25, u'_id': ObjectId('58c208bd6c42db2b20dc99e6'), u'name': u'cathy'}
{u'age': 32, u'_id': ObjectId('58c208bd6c42db2b20dc99e9'), u'name': u'fox'}


In [33]:
# age가 29 이상이거나 25인 doc
for doc in tmp.find({'$or' : [{'age' : {'$gte' : 29}}, {'age' : 25}]}):
    print doc

{u'age': 30, u'_id': ObjectId('58c208bd6c42db2b20dc99e5'), u'name': u'bob'}
{u'age': 25, u'_id': ObjectId('58c208bd6c42db2b20dc99e6'), u'name': u'cathy'}
{u'age': 32, u'_id': ObjectId('58c208bd6c42db2b20dc99e9'), u'name': u'fox'}


* nor
 - not or

In [34]:
for doc in tmp.find({'$nor' : [{'age' : {'$gte' : 29}}, {'age' : 25}]}):
    print doc

{u'age': 20, u'_id': ObjectId('58c208bd6c42db2b20dc99e4'), u'name': u'aaron'}
{u'age': 27, u'_id': ObjectId('58c208bd6c42db2b20dc99e7'), u'name': u'david'}
{u'age': 28, u'_id': ObjectId('58c208bd6c42db2b20dc99e8'), u'name': u'erick'}
{u'_id': ObjectId('58c208bd6c42db2b20dc99ea'), u'name': u'hmm'}


* in, nin
- mysql에서의 in과 같다. 그 필드 안의 것만 가져온다.

In [35]:
# age가 해당 리스트안에 존재하는 원소와 일치할 떄
for doc in tmp.find({'age' : {'$in' : [20, 21, 25, 27]}}):
    print doc

{u'age': 20, u'_id': ObjectId('58c208bd6c42db2b20dc99e4'), u'name': u'aaron'}
{u'age': 25, u'_id': ObjectId('58c208bd6c42db2b20dc99e6'), u'name': u'cathy'}
{u'age': 27, u'_id': ObjectId('58c208bd6c42db2b20dc99e7'), u'name': u'david'}


In [36]:
# nin : not in
for doc in tmp.find({'age' : {'$nin' : [20, 21, 25, 27]}}):
    print doc

{u'age': 30, u'_id': ObjectId('58c208bd6c42db2b20dc99e5'), u'name': u'bob'}
{u'age': 28, u'_id': ObjectId('58c208bd6c42db2b20dc99e8'), u'name': u'erick'}
{u'age': 32, u'_id': ObjectId('58c208bd6c42db2b20dc99e9'), u'name': u'fox'}
{u'_id': ObjectId('58c208bd6c42db2b20dc99ea'), u'name': u'hmm'}


* skip, limit
 - skip(n) : 검색 결과 n개만큼 건너뜀
 - limit(n) : 검색 결과 n개로 제한

In [37]:
for doc in tmp.find({'age' : {'$nin' : [20, 21, 25, 27]}}).skip(3):
    print doc

{u'_id': ObjectId('58c208bd6c42db2b20dc99ea'), u'name': u'hmm'}


In [38]:
for doc in tmp.find({'age' : {'$nin' : [20, 21, 25, 27]}}).limit(3):
    print doc

{u'age': 30, u'_id': ObjectId('58c208bd6c42db2b20dc99e5'), u'name': u'bob'}
{u'age': 28, u'_id': ObjectId('58c208bd6c42db2b20dc99e8'), u'name': u'erick'}
{u'age': 32, u'_id': ObjectId('58c208bd6c42db2b20dc99e9'), u'name': u'fox'}


In [39]:
# skip limit 동시 사용 가능
for doc in tmp.find({'age' : {'$nin' : [20, 21, 25, 27]}}).skip(1).limit(3):
    print doc

{u'age': 28, u'_id': ObjectId('58c208bd6c42db2b20dc99e8'), u'name': u'erick'}
{u'age': 32, u'_id': ObjectId('58c208bd6c42db2b20dc99e9'), u'name': u'fox'}
{u'_id': ObjectId('58c208bd6c42db2b20dc99ea'), u'name': u'hmm'}


* list, embedded document 검색

* embedded document 검색
 - document depth를 .연산자를 통하여 접근 가능

In [51]:
# query for embedded doc
for doc in movie.find({'datetime.year' : '2003'}):
    print doc
    

{u'title': u'\uc2e4\ubbf8\ub3c4', u'castings': [u'\uc124\uacbd\uad6c', u'\uc548\uc131\uae30'], u'_id': ObjectId('58c206c56c42db2b20dc99e0'), u'datetime': {u'year': u'2003', u'month': 3}}
{u'title': u'\uc2e4\ubbf8\ub3c4', u'castings': [u'\uc124\uacbd\uad6c', u'\uc548\uc131\uae30'], u'_id': ObjectId('58c207186c42db2b20dc99e3'), u'datetime': {u'year': u'2003', u'month': 3}}


* list 검색

In [52]:
# 리스트 원소에 이정재를 포함하는 경우 전부 반환
for doc in movie.find({'castings' : '이정재'}):
    print doc['title']

암살
암살


In [44]:
# 원소에 이정재와 전지현을 포함하는 경우
for doc in movie.find({'castings' : '이정재', 'castings' : '전지현'}): # ,는 and
    print doc['title']

암살
암살


In [53]:
# 원소에 이정재를 또는 안성기를 포함하는 경우
for doc in movie.find({'$or' : [{'castings' : '이정재'}, {'castings' : '안성기'}]}): # '또는'으로 묶었으므로
    print doc['title']

암살
실미도
암살
실미도


* 리스트로 비교하기 위해서는 순서와 값이 정확히 일치해야 함

In [46]:
for doc in movie.find({'castings' : ['이정재', '전지현', '하정우']}):
    print doc['title']

암살
암살


In [47]:
# 결과 없음
for doc in movie.find({'castings' : ['이정재', '전지현']}): # 부분집합이어도 안됨. 정확하게 매칭이 되어야함.
    print doc['title']

In [48]:
# 결과 없음
for doc in movie.find({'castings' : ['이정재', '하정우', '전지현']}):
    print doc['title']

* 순서에 관계 없이 찾고자 하는 경우

In [54]:
# 이정재, 하정우, 전지현이 모두 있는 경우 검색
for doc in movie.find({'castings' : {'$all' :  ['이정재', '하정우', '전지현'] }} ): # 순서 상관없음
    print doc['title']
# all 뒤에는 무조건 list가 옴. 원소가 하나여도 list로 해줘야함.

암살
암살


In [55]:
for doc in movie.find({'castings' : {'$all' :  ['하정우', '전지현'] }} ):
    print doc['title']
    

암살
암살


* elemMatch
 - 적어도 한개 이상의 원소가 복수개의 조건을 동시에 만족하는 경우
 - 조건이 한개인 경우는 사용하지 않음

In [56]:
score = mongo.test.score

In [57]:
score.insert_many([
{ '_id': 1, 'results': [ 82, 85, 88 ] },
{ '_id': 2, 'results': [ 75, 88, 89 ] }])

In [58]:
# 각 원소별로 하나라도 만족하는 경우
for doc in score.find({'results' : { '$gte': 80, '$lt': 85 }}): #여기서는 and가 아니다.
    print doc

{u'_id': 1, u'results': [82, 85, 88]}
{u'_id': 2, u'results': [75, 88, 89]}


In [63]:
# 적어도 한개의 원소(82) 가 80 <= 82 <= 85를 만족함
for doc in score.find({'results' :  {'$elemMatch' : { '$gte': 80, '$lt': 85 }}}): #elements를 사용하면 and로 묶어준다. 80이상 85미만
    print doc

{u'_id': 1, u'results': [82, 85, 88]}


* 인덱스를 특정하여 검색하기

In [64]:
# castings의 1번째 원소가 전지현인 것 검색
for doc in movie.find({'castings.1' : '전지현'} ):
    print doc['title']

암살
암살


In [65]:
# castings의 2번째 원소가 전지현인 것 검색
# 결과 없음
for doc in movie.find({'castings.2' : '전지현'} ):
    print doc['title']

* 리스트의 원소의 개수를 기준으로 검색

In [62]:
for doc in movie.find({'castings' : {'$size' : 3}} ):
    print doc['title']

암살
암살


In [66]:
# $size의 경우, 범위 지정 불가
# 범위를 지정하기 위해서는 별도의 field를 두고, 리스트의 개수를 업데이트 해야함
for doc in movie.find({'castings' : {'$size' : 3}} ):
    print doc['title']
    
for doc in movie.find({'castings' : {'$size' : {'$gt' : 3}}} ): # 이거 안됨. 별도의 필드를 만들어야 함.
    print doc['title'] 

암살
암살


OperationFailure: $size needs a number

### Update 예제

In [70]:
# update_one
mongo = MongoClient(mongo_server, 27017)
actor = mongo.cine21.actor

result = actor.update_one({'name' : '송강호'}, {'$set' : {'name' : '송광호'}}) # 맨 처음 것만 바꿈

print result
print result.modified_count

0


In [71]:
# update_many로 매칭되는 모든 doc 업데이트
result = actor.update_many({'name' : '송강호'}, {'$set' : {'name' : '송광호'}})
print result
print result.modified_count

0


* upsert option
 - update method의 upsert parameter가 True로 전달하면, 매칭되는 document를 찾지 못한 경우, insert 수행

In [72]:
result = actor.update_many({'name' : '송광호'}, {'$set' : {'name' : '오바마'}}, upsert=True) #못찾으면 추가. udate + insert
print result
print result.modified_count

result = actor.update_many({'name' : '클린턴'}, {'$set' : {'name' : '힐러리'}}, upsert=True)
print result
print result.upserted_id # 새로 삽입된 객체의 아이디
print result.modified_count

0
58c2170b1d543988885bb7bb
0


In [73]:
result = actor.update_one({'name' : '힐러리'}, 
                            {'$set' : {'name' : '캐서린', 'age' : 30}}, upsert=True)

### Delete 예제 

In [33]:
result = actor.delete_one({'name' : '캐서린'})
print result.deleted_count

result = actor.delete_many({'name' : '오바마'})
print result.deleted_count

1
4


* wget
 + 웹상의 파일을 다운로드 하는 프로그램
 + e.g) wget https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json

* mongoimport 
  - 파일로부터 db 생성
  - mongoimport --db test --collection zip --drop --file zips.json
  
* mongoexport 
  - db의 데이터를 파일에 백업
  - mongoexport --db test --collection news -o news.json

* mongodb index
 - https://docs.mongodb.com/manual/indexes/
 - SQL index와 개념적으로 동일
 - 즉, 검색을 더 빠르게 수행하고자 만드는 추가적인 data structure
 - index가 없으면 모든 collection의 document를 scan해야함

db.collection.createIndex({field1 : 1, field2 : 1 ...})

키의 값에는 1, -1 둘중의 하나가 올 수 있다.
1 : 순방향(ASCENDING), -1 : 역방향(DESCENDING)

* single field index
 - 한개의 필드에만 인덱스를 생성하는 경우
 - 방향은 중요하지 않음
 - db.student.createIndex({'name' : 1}) 이나 db.student.createIndex({'name' : -1})
 - 위의 두 인덱스 모두, query에 동일한 영향을 미침

* compound index
 - 복수개의 필드에 인덱스 생성 ex) score에도 걸고, name에도 걸고
 - 인덱스의 순서, 방향이 중요함
 - 즉, 맨처음 나타나틑 필드를 제외하고는 인덱스의 방향이 중요
 
 - 인덱스 방향의 중요성
   - createIndex({a : 1, b : -1})로 생성한 경우
   - db.x.find({a : 1, b : -1}), db.x.find({a : -1, b : 1}) 의 쿼리만 효과를 봄
   - db.x.find({a : 1, b : 1}), db.x.find({a : -1, b : -1}) 의 쿼리는 효과 없음
   
 - prefix
   - createIndex({a : 1, b : 1, c : 1, d : 1})로 생성한 경우
   - 효과 있는 쿼리
     - {a : 1} 
     - {a : 1, b : 1} 
     - {a : 1, b : 1, c : 1} 
     - {a : 1, b : 1, c : 1, d : 1}
   - 효과 없는 쿼리
     - {a : 1, c : 1}
     - {a : 1, b : 1, d : 1}
     - {b : 1, c : 1}
     - etc..
 - 즉, prefix를 갖는 인덱스를 중복하여 만들 필요가 없다.
 - 또한 복합인덱스의 경우 필드의 개수가 31개를 넘을 수 없다.
   

* text index
 - https://docs.mongodb.com/manual/reference/operator/query/text/#text-query-operator-behavior
 - text 필드의 검색을 위한 인덱스
 - createIndex({field : 'text'})로 생성

In [2]:
# test data 삽입

db.articles.insert(
   [
     { _id: 1, subject: "coffee", author: "xyz", views: 50 },
     { _id: 2, subject: "Coffee Shopping", author: "efg", views: 5 },
     { _id: 3, subject: "Baking a cake", author: "abc", views: 90  },
     { _id: 4, subject: "baking", author: "xyz", views: 100 },
     { _id: 5, subject: "Café Con Leche", author: "abc", views: 200 },
     { _id: 6, subject: "Сырники", author: "jkl", views: 80 },
     { _id: 7, subject: "coffee and cream", author: "efg", views: 10 },
     { _id: 8, subject: "Cafe con Leche", author: "xyz", views: 10 }
     { _id: 9, subject: "coffees", author: "xyz", views: 10 },
     { _id: 10, subject: "coffee1", author: "xyz", views: 10 },
   ]
)

# text 인덱스 생성 
db.articles.createIndex({'subject' : 'text'})


# query (stemmed word 검색)
# 기본적으로 대소문자 구별하지 않음
db.articles.find({'$text' : {'$search' : 'coffee'}})

# 대소문자 구별
db.articles.find({'$text' : {'$search' : 'coffee', '$caseSensitive' : true}})

# 띄어쓰기 있는 문자열
# 각 단어를 검색한 결과를 or로 연결
db.articles.find({'$text' : {'$search' : 'coffee cake cream'}})

# 매칭 단어 검색
# escaped "를 사용
db.articles.find({'$text' : {'$search' : '\"coffee and\"'}})

# 정규표현식
db.articles.find({'subject' : {'$regex' : /cof/}})
db.articles.find({'subject' : {'$regex' : 'cof.+'}})

SyntaxError: invalid syntax (<ipython-input-2-5abc2ae380d0>, line 13)

* index 삭제
 - db.x.dropIndex({'field' : 1}) 와 같이 생성한 문서 그대로 전달하여 삭제

* index 확인
  - db.x.getIndexes()로 생성된 인덱스 확인 가능

* mongodb aggregation
  - 저장되어 있는 복수개의 documents grouping, filtering 등 다양한 연산을 적용하여 계산된 결과를 반환하는 것
  - aggregation 파이프라인을 사용
  - https://docs.mongodb.com/manual/meta/aggregation-quick-reference/#aggregation-expressions
  - e.g) filtering, like operation, transforming
  - ![aggregation](https://docs.mongodb.com/v3.2/_images/aggregation-pipeline.png)
    이미지 출처 - https://docs.mongodb.com/manual/aggregation/#aggregation-framework

 * 예제1) 미국 zip code 데이터 이용하기
  - http://media.mongodb.org/zips.json
  
  

 * 1000만 이상의 state만 가져오기
   - group : 문자 그대로 grouping의 역할을 함.
   - match : 조건에 맞는 documents만 필터링 함

 db.zip.aggregate([
             {$group : {_id : '$state', totalPop : {$sum : '$pop'}}}, 
             {$match : {totalPop : {$gte : 10 * 1000 * 1000}}}
])

* 주별 평균 city 인구 수 구하기

db.zip.aggregate([ 
            { $group: { _id: { state: "$state"}, pop: { $sum: "$pop" }}}, 
            { $group: { _id: "$_id.state", avgCityPop: { $avg: "$pop" }}} 
] )

* 주별 최대, 최소 인구 수 city 구하기
  - project : 문서 필드 추가, 삭제 이름 변경 등 각 문서별로 적용

In [ ]:
db.zip.aggregate( [
   { $group:
      {
        _id: { state: "$state", city: "$city" },
        pop: { $sum: "$pop" }
      }
   },
   { $sort: { pop: 1 } },
   { $group:
      {
        _id : "$_id.state",
        biggestCity:  { $last: "$_id.city" },
        biggestPop:   { $last: "$pop" },
        smallestCity: { $first: "$_id.city" },
        smallestPop:  { $first: "$pop" }
      }
   },

  { $project:
    { _id: 0,
      state: "$_id",
      biggestCity:  { name: "$biggestCity",  pop: "$biggestPop" },
      smallestCity: { name: "$smallestCity", pop: "$smallestPop" }
    }
  }
] )

* pymongo aggregation
  - 위의 예제들을 그대로 pymongo로 실행

In [23]:
from pymongo import MongoClient
mongo = MongoClient(mongo_server, 27017)

col = mongo.test.zip

'''{$group : {_id : '$state', totalPop : {$sum : '$pop'}}}, 
   {$match : {totalPop : {$gte : 10 * 1000 * 1000}}}'''

pipelines = []
pipelines.append({'$group' : {'_id' : '$state', 'totalPop' : {'$sum' : '$pop'}}})
pipelines.append({'$match' : {'totalPop' : {'$gte' : 10 * 1000 * 1000}}})

result = col.aggregate(pipelines)
print type(result)
for doc in result:
    print doc

<class 'pymongo.command_cursor.CommandCursor'>
{u'_id': u'CA', u'totalPop': 29754890}
{u'_id': u'FL', u'totalPop': 12686644}
{u'_id': u'PA', u'totalPop': 11881643}
{u'_id': u'NY', u'totalPop': 17990402}
{u'_id': u'OH', u'totalPop': 10846517}
{u'_id': u'IL', u'totalPop': 11427576}
{u'_id': u'TX', u'totalPop': 16984601}


In [24]:
mongo = MongoClient(mongo_server, 27017)

col = mongo.test.zip

'''{ $group: { _id: { state: "$state", city: "$city" }, pop: { $sum: "$pop" }}}, 
   { $group: { _id: "$_id.state", avgCityPop: { $avg: "$pop" }}}'''

pipelines = []
pipelines.append({ '$group': { '_id': { 'state': "$state", 'city': "$city" }, 'pop': { '$sum': "$pop" }}})
pipelines.append({ '$group': { '_id': "$_id.state", 'avgCityPop': { '$avg': "$pop" }}})

result = col.aggregate(pipelines)
print type(result)
for doc in result:
    print doc

<class 'pymongo.command_cursor.CommandCursor'>
{u'_id': u'DC', u'avgCityPop': 303450.0}
{u'_id': u'DE', u'avgCityPop': 14481.91304347826}
{u'_id': u'VT', u'avgCityPop': 2315.8765432098767}
{u'_id': u'ME', u'avgCityPop': 3006.4901960784314}
{u'_id': u'NJ', u'avgCityPop': 15775.89387755102}
{u'_id': u'MT', u'avgCityPop': 2593.987012987013}
{u'_id': u'CA', u'avgCityPop': 27756.42723880597}
{u'_id': u'KS', u'avgCityPop': 3819.884259259259}
{u'_id': u'MO', u'avgCityPop': 5672.195338512764}
{u'_id': u'NH', u'avgCityPop': 5232.320754716981}
{u'_id': u'OK', u'avgCityPop': 6155.743639921722}
{u'_id': u'NE', u'avgCityPop': 3034.882692307692}
{u'_id': u'CO', u'avgCityPop': 9981.075757575758}
{u'_id': u'NC', u'avgCityPop': 10622.815705128205}
{u'_id': u'TX', u'avgCityPop': 13775.02108678021}
{u'_id': u'ND', u'avgCityPop': 1645.0309278350514}
{u'_id': u'NY', u'avgCityPop': 13131.680291970803}
{u'_id': u'NM', u'avgCityPop': 5872.360465116279}
{u'_id': u'IL', u'avgCityPop': 9954.334494773519}
{u'_id'

In [25]:
mongo = MongoClient(mongo_server, 27017)

col = mongo.test.zip

'''{ $group:
      {
        _id: { state: "$state", city: "$city" },
        pop: { $sum: "$pop" }
      }
   },
   { $sort: { pop: 1 } },
   { $group:
      {
        _id : "$_id.state",
        biggestCity:  { $last: "$_id.city" },
        biggestPop:   { $last: "$pop" },
        smallestCity: { $first: "$_id.city" },
        smallestPop:  { $first: "$pop" }
      }
   },

  { $project:
    { _id: 0,
      state: "$_id",
      biggestCity:  { name: "$biggestCity",  pop: "$biggestPop" },
      smallestCity: { name: "$smallestCity", pop: "$smallestPop" }
    }
  }'''

pipelines = []
pipelines.append({ '$group': { '_id': { 'state': "$state", 'city': "$city" }, 'pop': { '$sum': "$pop" }}})
pipelines.append({ '$sort': { 'pop': 1 } })
pipelines.append({ '$group' : {'_id' : "$_id.state", 'biggestCity':  { '$last': "$_id.city" }, 'biggestPop' : { '$last': "$pop" }, 'smallestCity': { '$first': "$_id.city" }, 'smallestPop':  { '$first': "$pop" }}})
pipelines.append({ '$project' : { '_id' : 0, 'state': "$_id", 'biggestCity':  { 'name': "$biggestCity",  'pop': "$biggestPop" }, 'smallestCity': { 'name': "$smallestCity", 'pop': "$smallestPop" }}})

result = col.aggregate(pipelines)
print type(result)
for doc in result:
    print doc

<class 'pymongo.command_cursor.CommandCursor'>
{u'state': u'DE', u'biggestCity': {u'name': u'NEWARK', u'pop': 111674}, u'smallestCity': {u'name': u'BETHEL', u'pop': 108}}
{u'state': u'MS', u'biggestCity': {u'name': u'JACKSON', u'pop': 204788}, u'smallestCity': {u'name': u'CHUNKY', u'pop': 79}}
{u'state': u'RI', u'biggestCity': {u'name': u'CRANSTON', u'pop': 176404}, u'smallestCity': {u'name': u'CLAYVILLE', u'pop': 45}}
{u'state': u'MO', u'biggestCity': {u'name': u'SAINT LOUIS', u'pop': 397802}, u'smallestCity': {u'name': u'BENDAVIS', u'pop': 44}}
{u'state': u'GA', u'biggestCity': {u'name': u'ATLANTA', u'pop': 609591}, u'smallestCity': {u'name': u'FORT STEWART', u'pop': 0}}
{u'state': u'MN', u'biggestCity': {u'name': u'MINNEAPOLIS', u'pop': 344719}, u'smallestCity': {u'name': u'JOHNSON', u'pop': 12}}
{u'state': u'VT', u'biggestCity': {u'name': u'BURLINGTON', u'pop': 39127}, u'smallestCity': {u'name': u'UNIV OF VERMONT', u'pop': 0}}
{u'state': u'NM', u'biggestCity': {u'name': u'ALBUQUERQ

* 연습문제) 
 0. 위의 예제 1번을 직접 python으로 구현해보세요.
 1. import한 restaurant 데이터를 가지고 각 borough 마다 레스토랑이 몇 개있는지 계산하시오.
 2. address의 zipcode별로 'Brazilian' 레스토랑이 몇개 있는지 계산하시오.
 3. 각자의 데이터에 필요한 aggregation을 진행해보세요. (자유롭게 질문하시면 됩니다.)
 
 
 
 
 
 
 
 
 
 
 
 
 